In [1]:
# Math and IO
import numpy as np
from scipy.signal import medfilt
from astropy.io import fits
from glob import glob
import os
import sys

# Plotting
import matplotlib.pyplot as plt
import petitRADTRANS.retrieval.plot_style

# Rebin Spectra
from spectres import spectres


Using pRT Plotting style!


In [ ]:
def plot_hdu(fig,ax,hdu,flux_scale = 1.0, err_scale = 1.0,color='k',snr=True,count=-1):
    spec = hdu['SPECTRUM'].data
    flux = spec['FLUX']*flux_scale

    err = np.diag(spec['COVARIANCE'])**0.5
    corr = spec['CORRELATION']
    t_err = np.sqrt(np.diag(spec['COVARIANCE']))*err_scale

    if snr:
        print(count,np.mean(flux/(t_err)),np.min(flux/(t_err)),np.max(flux/(t_err)))
    ax.errorbar(spec['WAVELENGTH'],flux,t_err,c=color,alpha=0.4)
    return fig,ax
def plot_hdul(fig,ax,hdul,flux_scale=1.0,err_scale = 1.0, snr=True,cmap = 'PuRd_r'):
    indices = np.linspace(0.0,0.5,len(hdul))
    cmap = plt.get_cmap(cmap)(indices)
    if snr:
        print(hdul[0][0].header['OBJECT'])
    for i,hdu in enumerate(hdul):
        plot_hdu(fig,ax,hdu,flux_scale,err_scale,cmap[i],snr,i)
        
def save_one_fits(hdr, wavelength, spectrum, covariance, correlation, instrument, reduction, obj = "", contrast = None, cont_cov = None):
    error = np.sqrt(np.dot(covariance,np.ones_like(spectrum)))
    wvs = fits.open("wavelength.fits")[0].data
    primary_hdu = fits.PrimaryHDU([])
    primary_hdu.header = hdr
    primary_hdu.header['OBJECT'] = obj
    primary_hdu.header['SNR'] = np.mean(spectrum/error)
    c1 = fits.Column(name = "WAVELENGTH", array = wavelength, format = 'D',unit = "micron")
    c2 = fits.Column(name = "FLUX", array = spectrum, format = 'D',unit = "W/m2/micron")
    c3 = fits.Column(name = "COVARIANCE", array = covariance, format = str(covariance.shape[0])+'D',unit = "[W/m2/micron]^2")
    c4 = fits.Column(name = "ERROR", array = error, format ='D',unit = "[W/m2/micron]")
    c5 = fits.Column(name = "CORRELATION", array = correlation, format =  str(correlation.shape[0])+'D',unit = " - ")
    columns = [c1,c2,c3,c4,c5]
    if contrast is not None:
        c6 = fits.Column(name = "CONTRAST", array = contrast, format = 'D',unit = " - ")
        c7 = fits.Column(name = "COVARIANCE_CONTRAST", array = cont_cov, format = str(cont_cov.shape[0])+'D',unit = " - ^2")
        columns.extend([c6,c7])
    table_hdu = fits.BinTableHDU.from_columns(columns,name = 'SPECTRUM')
    hdul = fits.HDUList([primary_hdu,table_hdu])

    outstring = data_dir + obj + "_" + instrument + "_" + reduction
    outstring += "_spectrum.fits"
    #print(outstring)
    hdul.writeto(outstring,overwrite=True,checksum=True,output_verify='exception')
    return

def save_to_fits_final(hdul, ind, instrument, reduction, obj = "", scaleflux=1.0, scalerr=1.0):
    spec = hdul[ind]['SPECTRUM'].data
    wvs = spec['WAVELENGTH']
    flux = spec['FLUX'] * scaleflux
    covar = spec['COVARIANCE']*scalerr
    corr = spec['CORRELATION']
    cont = None
    cont_cov = None
    try:
        cont = spec["CONTRAST"]
        cont_cov = spec["COVARIANCE_CONTRAST"]
    except:
        pass
    save_one_fits(hdul[ind][0].header,wvs,flux,covar,corr, instrument, reduction, obj, cont, cont_cov)
